In [1]:
import urllib2
from bs4 import BeautifulSoup
from urlparse import urljoin
from pysqlite2 import dbapi2 as sqlite
import re

In [2]:
# Create a list of words to ignore
ignorewords={'the':1,'of':1,'to':1,'and':1,'a':1,'in':1,'is':1,'it':1}

In [3]:
class crawler:
    # Initialize the crawler with the name of database
    def __init__(self,dbname):
        self.con=sqlite.connect(dbname)

    def __del__(self):
        self.con.close()

    def dbcommit(self):
        self.con.commit()

    # Auxilliary function for getting an entry id and adding 
    # it if it's not present
    def getentryid(self,table,field,value,createnew=True):
        cur = self.con.execute("select rowid from %s where %s='%s'" % (table,field,value))
        res = cur.fetchone()
        if res == None:
            cur = self.con.execute("insert into %s (%s) values ('%s')" % (table,field,value))
            return cur.lastrowid
        else:
            return res[0] 

    # Index an individual page
    def addtoindex(self,url,soup):
        if self.isindexed(url):
            print 'already had indexing :' + url
            return
        print 'Indexing ' + url
  
        # Get the individual words
        try:
            text = self.gettextonly(soup)
            #print 'text:',text
        except:
            print "gettextonly() error" 
        #
        try:
            words = self.separatewords(text)
            #print 'words:',words
        except Exception,ex:
            print "separatewords() error",ex
        #    
        print 'len(text):',len(text)
        print 'len(words):',len(words)        
        # Get the URL id
        urlid = self.getentryid('urllist','url',url)
    
        # Link each word to this url
        for i in range(len(words)):
            word = words[i]
            if word in ignorewords: continue
            wordid = self.getentryid('wordlist','word',word)
            self.con.execute("insert into wordlocation(urlid,wordid,location) values (%d,%d,%d)" % (urlid,wordid,i))


  
    # Extract the text from an HTML page (no tags)
    def gettextonly(self,soup):
        v = soup.string
        #
        if v == None:   
            c = soup.contents
            #print 'soup.contents:',c
            resulttext = ''
            for t in c:
                subtext = self.gettextonly(t)
                resulttext += subtext + '\n'
            return resulttext
        else:
            #print 'soup.string:',v
            return v.strip()
        
    # Seperate the words by any non-whitespace character
    def separatewords(self,text):
        splitter=re.compile('\\W*')
        return [s.lower() for s in splitter.split(text) if s!='']

    
    # Return true if this url is already indexed
    def isindexed(self,url):
        u = self.con.execute("select rowid from urllist where url='%s'" % url).fetchone()
        if u != None:
            #检查它是否已经被检索过了
            v = self.con.execute('select * from wordlocation where urlid=%d ' % u[0]).fetchone()
            if v != None: return True
        return False
  
    # Add a link between two pages
    def addlinkref(self,urlFrom,urlTo,linkText):
        words = self.separatewords(linkText)
        fromid = self.getentryid('urllist','url',urlFrom)
        toid = self.getentryid('urllist','url',urlTo)
        if fromid == toid: return
        cur = self.con.execute("insert into link(fromid,toid) values (%d,%d)" % (fromid,toid))
        linkid = cur.lastrowid
        for word in words:
            if word in ignorewords: continue
            wordid = self.getentryid('wordlist','word',word)
            self.con.execute("insert into linkwords(linkid,wordid) values (%d,%d)" % (linkid,wordid))

    # Starting with a list of pages, do a breadth
    # first search to the given depth, indexing pages
    # as we go
    def crawl(self,pages,depth=2):
        for i in range(depth):
            newpages = {}
            for page in pages:
                try:
                    c = urllib2.urlopen(page)
                except Exception,ex:
                    print "Could not open %s" % page,ex
                    continue
                try:
                    soup = BeautifulSoup(c.read(),"lxml")
                    self.addtoindex(page,soup)
  
                    links = soup('a')
                    for link in links:
                        if ('href' in dict(link.attrs)):
                            #print 'href: ', link['href']
                            url = urljoin(page,link['href'])
                            #print 'url: ', url
                            if url.find("'") != -1 : 
                                print 'not find: ',url
                                continue
                            url = url.split('#')[0]  # remove location portion
                            if url[0:4] == 'http' and not self.isindexed(url):
                                newpages[url] = 1
                            linkText = self.gettextonly(link)
                            self.addlinkref(page,url,linkText)
                    
                    self.dbcommit()
                except Exception,ex:
                    print "Could not parse page %s" % page,ex
            print 'newpages len: ', len(newpages)
            pages = newpages
    
        
    # Create the database tables
    def createindextables(self):
        self.con.execute('create table urllist(url)')
        self.con.execute('create table wordlist(word)')
        self.con.execute('create table wordlocation(urlid,wordid,location)')
        self.con.execute('create table link(fromid integer,toid integer)')
        self.con.execute('create table linkwords(wordid,linkid)')
        self.con.execute('create index wordidx on wordlist(word)')
        self.con.execute('create index urlidx on urllist(url)')
        self.con.execute('create index wordurlidx on wordlocation(wordid)')
        self.con.execute('create index urltoidx on link(toid)')
        self.con.execute('create index urlfromidx on link(fromid)')
        self.dbcommit()

    def calculatepagerank(self,iterations=20):
        # clear out the current page rank tables
        self.con.execute('drop table if exists pagerank')
        self.con.execute('create table pagerank(urlid primary key,score)')
    
        # initialize every url with a page rank of 1
        for (urlid,) in self.con.execute('select rowid from urllist'):
            self.con.execute('insert into pagerank(urlid,score) values (%d,1.0)' % urlid)
        self.dbcommit()
    
        for i in range(iterations):
            print "Iteration %d" % (i)
            for (urlid,) in self.con.execute('select rowid from urllist'):
                pr = 0.15

                # Loop through all the pages that link to this one
                for (linker,) in self.con.execute('select distinct fromid from link where toid=%d' % urlid):
                    # Get the page rank of the linker
                    linkingpr = self.con.execute('select score from pagerank where urlid=%d' % linker).fetchone()[0]

                    # Get the total number of links from the linker
                    linkingcount = self.con.execute('select count(*) from link where fromid=%d' % linker).fetchone()[0]
                    pr += 0.85 * (linkingpr/linkingcount)
                
                #
                self.con.execute('update pagerank set score=%f where urlid=%d' % (pr,urlid))
            #
            self.dbcommit()        

In [4]:
pagelist = [ 'http://en.people.cn/']
crawlerObj = crawler('searchindex.db')

In [ ]:
crawlerObj.createindextables()

In [ ]:
crawlerObj.crawl(pagelist,depth=2)

In [6]:
crawlerObj.calculatepagerank()

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19


In [7]:
cur = crawlerObj.con.execute('select * from pagerank order by score desc')
for i in range(3): print cur.next()

(8, 0.293662)
(128, 0.293516)
(115, 0.290625)


In [5]:
[row for row in crawlerObj.con.execute('select urlid from wordlocation where wordid=200') ]

[(1,), (17,), (35,)]

In [16]:
class searcher:
    def __init__(self,dbname):
        self.con=sqlite.connect(dbname)

    def __del__(self):
        self.con.close()
        
    def getmatchrows(self,q):
        # Strings to build the query
        fieldlist='w0.urlid'
        tablelist=''  
        clauselist=''
        wordids=[]

        # Split the words by spaces
        words = q.split(' ')  
        tablenumber = 0

        for word in words:
            # Get the word ID
            wordrow = self.con.execute("select rowid from wordlist where word='%s'" % word).fetchone()
            if wordrow != None:
                wordid = wordrow[0]
                wordids.append(wordid)
                if tablenumber > 0:
                    tablelist += ','
                    clauselist += ' and '
                    clauselist += 'w%d.urlid = w%d.urlid and ' % (tablenumber - 1,tablenumber)
                fieldlist += ',w%d.location' % tablenumber
                tablelist += 'wordlocation w%d' % tablenumber      
                clauselist += 'w%d.wordid = %d' % (tablenumber,wordid)
                tablenumber+=1

        # Create the query from the separate parts
        fullquery = 'select %s from %s where %s' % (fieldlist,tablelist,clauselist)
        print fullquery
        cur = self.con.execute(fullquery)
        rows=[row for row in cur]

        return rows,wordids   
    
    def getscoredlist(self,rows,wordids):
        totalscores = dict([(row[0],0) for row in rows])

        # This is where we'll put our scoring functions
        weights = [(1.0,self.frequencyscore(rows)),
                   (1.0,self.locationscore(rows)),
                   (1.0,self.distancescore(rows)),
                   (1.0,self.inboundlinkscore(rows)),
                   (1.0,self.pagerankscore(rows)),
                   (1.0,self.linktextscore(rows,wordids)),
                   (5.0,self.nnscore(rows,wordids))]
        #
        for (weight,scores) in weights:
            for url in totalscores:
                totalscores[url] += weight * scores[url]
        return totalscores

    def geturlname(self,id):
        return self.con.execute("select url from urllist where rowid=%d" % id).fetchone()[0]

    def query(self,q):
        rows,wordids = self.getmatchrows(q)
        scores = self.getscoredlist(rows,wordids)
        rankedscores=[(score,url) for (url,score) in scores.items()]
        rankedscores.sort()
        rankedscores.reverse()
        for (score,urlid) in rankedscores[0:10]:
            print '%f\t%s' % (score,self.geturlname(urlid))
        return wordids,[r[1] for r in rankedscores[0:10]]

    def normalizescores(self,scores,smallIsBetter=0):
        vsmall = 0.00001 # Avoid division by zero errors
        if smallIsBetter:
            minscore = min(scores.values())
            return dict([(u,float(minscore)/max(vsmall,l)) for (u,l) in scores.items()])
        else:
            maxscore = max(scores.values())
            if maxscore == 0: maxscore = vsmall
            return dict([(u,float(c)/maxscore) for (u,c) in scores.items()])        
        
    def frequencyscore(self,rows):
        counts = dict([(row[0],0) for row in rows])
        for row in rows: counts[row[0]] += 1
        return self.normalizescores(counts)  
    
    def locationscore(self,rows):
        locations = dict([(row[0],1000000) for row in rows])
        for row in rows:
            loc = sum(row[1:])
            if loc < locations[row[0]]: locations[row[0]] = loc
    
        return self.normalizescores(locations,smallIsBetter=1) 
    
    def distancescore(self,rows):
        # If there's only one word, everyone wins!
        if len(rows[0]) <= 2: return dict([(row[0],1.0) for row in rows])

        # Initialize the dictionary with large values
        mindistance = dict([(row[0],1000000) for row in rows])

        for row in rows:
            dist = sum([abs(row[i]-row[i-1]) for i in range(2,len(row))])
            if dist < mindistance[row[0]]: mindistance[row[0]] = dist
                
        return self.normalizescores(mindistance,smallIsBetter=1)    

    def inboundlinkscore(self,rows):
        uniqueurls = dict([(row[0],1) for row in rows])
        inboundcount = dict([(u, self.con.execute('select count(*) from link where toid=%d' % u).fetchone()[0]) for u in uniqueurls])   
        return self.normalizescores(inboundcount)
    
    def pagerankscore(self,rows):
        pageranks=dict([(row[0],self.con.execute('select score from pagerank where urlid=%d' % row[0]).fetchone()[0]) for row in rows])
        maxrank = max(pageranks.values())
        normalizedscores = dict([(u,float(l)/maxrank) for (u,l) in pageranks.items()])
        return normalizedscores    
    
    def linktextscore(self,rows,wordids):
        vsmall = 0.00001 # Avoid division by zero errors
        linkscores = dict([(row[0],0) for row in rows])
        for wordid in wordids:
            cur = self.con.execute('select link.fromid,link.toid from linkwords,link where wordid=%d and linkwords.linkid=link.rowid' % wordid)
            for (fromid,toid) in cur:
                if toid in linkscores:
                    pr = self.con.execute('select score from pagerank where urlid=%d' % fromid).fetchone()[0]
                    linkscores[toid] += pr
        maxscore = max(max(linkscores.values()),vsmall)
        normalizedscores = dict([(u,float(l)/maxscore) for (u,l) in linkscores.items()])
        return normalizedscores  
    
    def nnscore(self,rows,wordids):
        # Get unique URL IDs as an ordered list
        urlids = [urlid for urlid in dict([(row[0],1) for row in rows])]
        nnres = mynet.getresult(wordids,urlids)
        scores = dict([(urlids[i],nnres[i]) for i in range(len(urlids))])
        return self.normalizescores(scores)    

In [14]:
searcherObj = searcher('searchindex.db')

In [ ]:
searcherObj.getmatchrows('prohibit')

In [ ]:
searcherObj.getmatchrows('prohibit business')

In [ ]:
searcherObj.query('prohibit business')

In [ ]:
searcherObj.query('prohibit business')

In [15]:
searcherObj.query('society business')

select w0.urlid,w0.location,w1.location from wordlocation w0,wordlocation w1 where w0.wordid = 63 and w0.urlid = w1.urlid and w1.wordid = 60
3.369600	http://en.people.cn/90882/index.html
3.192588	http://en.people.cn/business/index.html
3.015823	http://en.people.cn/index.html
2.945477	http://en.people.cn/90783/index.html
2.928196	http://www.ecns.cn

2.731780	http://beijingtoday.com.cn/
2.717072	http://en.people.cn/102775/index.html
2.645377	http://en.ce.cn/

2.571570	http://en.people.cn/90782/index.html
2.559845	http://en.people.cn/90777/index.html


In [10]:
searcherObj.geturlname(8)

u'http://www.people.com.cn/'

In [11]:
searcherObj.geturlname(128)

u'http://en.gmw.cn/'

In [12]:
searcherObj.geturlname(115)

u'http://beijingtoday.com.cn/'

In [18]:
searcherObj.query('business')

select w0.urlid,w0.location from wordlocation w0 where w0.wordid = 60
4.534850	http://en.people.cn/business/index.html
3.603527	http://www.ecns.cn

3.585932	http://www.chinadaily.com.cn/

3.492530	http://en.people.cn/102775/index.html
3.407496	http://en.people.cn/90783/index.html
3.314812	http://en.people.cn/index.html
3.292892	http://english.sina.com/
3.288656	http://en.ce.cn/

3.245058	http://en.people.cn/90882/index.html
3.174218	http://en.gmw.cn/


In [22]:
searcherObj.query('world bank')

select w0.urlid,w0.location,w1.location from wordlocation w0,wordlocation w1 where w0.wordid = 62 and w0.urlid = w1.urlid and w1.wordid = 4970
3.199759	http://en.gmw.cn/
3.019432	http://en.people.cn/business/index.html
2.697951	http://eng.taiwan.cn/

2.571934	http://en.ce.cn/

2.237768	http://en.people.cn/n3/2016/0306/c90000-9025834.html
2.200150	http://en.people.cn/business/n3/2016/0309/c90778-9027459.html
1.410790	http://en.people.cn/102775/209361/index.html
1.358100	http://en.people.cn/n3/2016/0310/c90000-9027804.html
1.261618	http://en.people.cn/business/n3/2016/0309/c90778-9027425.html
1.140786	http://en.people.cn/n3/2016/0310/c90000-9028220.html


In [32]:
from math import tanh
from pysqlite2 import dbapi2 as sqlite

def dtanh(y):
    return 1.0 - y*y

class searchnet:
    def __init__(self,dbname):
        self.con=sqlite.connect(dbname)
  
    def __del__(self):
        self.con.close()

    def maketables(self):
        self.con.execute('create table hiddennode(create_key)')
        self.con.execute('create table wordhidden(fromid,toid,strength)')
        self.con.execute('create table hiddenurl(fromid,toid,strength)')
        self.con.commit()
        
    def getstrength(self,fromid,toid,layer):
        if layer == 0: table = 'wordhidden'
        else: table='hiddenurl'
        res = self.con.execute('select strength from %s where fromid=%d and toid=%d' % (table,fromid,toid)).fetchone()
        if res == None: 
            if layer == 0: return -0.2
            if layer == 1: return 0
        return res[0]

    def setstrength(self,fromid,toid,layer,strength):
        if layer == 0: table='wordhidden'
        else: table = 'hiddenurl'
        res = self.con.execute('select rowid from %s where fromid=%d and toid=%d' % (table,fromid,toid)).fetchone()
        if res == None: 
            self.con.execute('insert into %s (fromid,toid,strength) values (%d,%d,%f)' % (table,fromid,toid,strength))
        else:
            rowid = res[0]
            self.con.execute('update %s set strength=%f where rowid=%d' % (table,strength,rowid))    
           
    def generatehiddennode(self,wordids,urls):
        if len(wordids) > 3: return None
        # Check if we already created a node for this set of words
        sorted_words = [str(id) for id in wordids]
        sorted_words.sort()
        createkey = '_' . join(sorted_words)
        res = self.con.execute("select rowid from hiddennode where create_key='%s'" % createkey).fetchone()

        # If not, create it
        if res == None:
            cur = self.con.execute("insert into hiddennode (create_key) values ('%s')" % createkey)
            hiddenid = cur.lastrowid
            # Put in some default weights
            for wordid in wordids:
                self.setstrength(wordid,hiddenid,0,1.0/len(wordids))
            #
            for urlid in urls:
                self.setstrength(hiddenid,urlid,1,0.1)
            self.con.commit()    
            
    def getallhiddenids(self,wordids,urlids):
        l1 = {}
        for wordid in wordids:
            cur = self.con.execute('select toid from wordhidden where fromid=%d' % wordid)
            for row in cur: l1[row[0]] = 1
        for urlid in urlids:
            cur = self.con.execute('select fromid from hiddenurl where toid=%d' % urlid)
            for row in cur: l1[row[0]] = 1
        return l1.keys() 
    
    def setupnetwork(self,wordids,urlids):
        # value lists
        self.wordids = wordids
        self.hiddenids = self.getallhiddenids(wordids,urlids)
        self.urlids = urlids
 
        # node outputs
        self.ai = [1.0] * len(self.wordids)
        self.ah = [1.0] * len(self.hiddenids)
        self.ao = [1.0] * len(self.urlids)
        
        # create weights matrix
        self.wi = [[self.getstrength(wordid,hiddenid,0)  for hiddenid in self.hiddenids] for wordid in self.wordids]
        self.wo = [[self.getstrength(hiddenid,urlid,1) for urlid in self.urlids] for hiddenid in self.hiddenids]    
        
    def feedforward(self):
        # the only inputs are the query words
        for i in range(len(self.wordids)):
            self.ai[i] = 1.0

        # hidden activations
        for j in range(len(self.hiddenids)):
            sum = 0.0
            for i in range(len(self.wordids)):
                sum = sum + self.ai[i] * self.wi[i][j]
            self.ah[j] = tanh(sum)

        # output activations
        for k in range(len(self.urlids)):
            sum = 0.0
            for j in range(len(self.hiddenids)):
                sum = sum + self.ah[j] * self.wo[j][k]
            self.ao[k] = tanh(sum)

        return self.ao[:]    
    
    def getresult(self,wordids,urlids):
        self.setupnetwork(wordids,urlids)
        return self.feedforward()    
    
    def backPropagate(self, targets, N=0.5):
        # calculate errors for output
        output_deltas = [0.0] * len(self.urlids)
        for k in range(len(self.urlids)):
            error = targets[k] - self.ao[k]
            output_deltas[k] = dtanh(self.ao[k]) * error

        # calculate errors for hidden layer
        hidden_deltas = [0.0] * len(self.hiddenids)
        for j in range(len(self.hiddenids)):
            error = 0.0
            for k in range(len(self.urlids)):
                error = error + output_deltas[k]*self.wo[j][k]
            hidden_deltas[j] = dtanh(self.ah[j]) * error

        # update output weights
        for j in range(len(self.hiddenids)):
            for k in range(len(self.urlids)):
                change = output_deltas[k]*self.ah[j]
                self.wo[j][k] = self.wo[j][k] + N*change

        # update input weights
        for i in range(len(self.wordids)):
            for j in range(len(self.hiddenids)):
                change = hidden_deltas[j]*self.ai[i]
                self.wi[i][j] = self.wi[i][j] + N*change  
                
    def trainquery(self,wordids,urlids,selectedurl): 
        # generate a hidden node if necessary
        self.generatehiddennode(wordids,urlids)

        self.setupnetwork(wordids,urlids)      
        self.feedforward()
        targets=[0.0]*len(urlids)
        targets[urlids.index(selectedurl)] = 1.0
        error = self.backPropagate(targets)
        self.updatedatabase()   
        
    def updatedatabase(self):
        # set them to database values
        for i in range(len(self.wordids)):
            for j in range(len(self.hiddenids)):
                self.setstrength(self.wordids[i],self. hiddenids[j],0,self.wi[i][j])
        for j in range(len(self.hiddenids)):
            for k in range(len(self.urlids)):
                self.setstrength(self.hiddenids[j],self.urlids[k],1,self.wo[j][k])
        self.con.commit()        

In [33]:
mynet = searchnet('nn.db')

In [ ]:
mynet.maketables()

In [26]:
wWor1d,wRivar,wBank = 101,102,103
uWor1dBank,uRivar,uEarth  = 201,202,203
mynet.generatehiddennode([wWor1d,wBank],[uWor1dBank,uRivar,uEarth])

In [27]:
for c in mynet.con.execute( 'select * from wordhidden'): print c

(101, 1, 0.5)
(103, 1, 0.5)


In [28]:
for c in mynet.con.execute( 'select * from hiddenurl'): print c

(1, 201, 0.1)
(1, 202, 0.1)
(1, 203, 0.1)


In [31]:
mynet.getresult([wWor1d,wBank],[uWor1dBank,uRivar,uEarth])

[0.07601250837541615, 0.07601250837541615, 0.07601250837541615]

In [36]:
mynet.trainquery([wWor1d,wBank],[uWor1dBank,uRivar,uEarth],uWor1dBank)

In [35]:
mynet.getresult([wWor1d,wBank],[uWor1dBank,uRivar,uEarth])

[0.3350632467125332, 0.055127057492088, 0.055127057492088]

In [37]:
mynet.getresult([wWor1d,wBank],[uWor1dBank,uRivar,uEarth])

[0.5016308751693285, 0.040561698154791534, 0.040561698154791534]

In [40]:
allurls = [uWor1dBank,uRivar,uEarth]
for i in range(30):
    mynet.trainquery([wWor1d,wBank],allurls,uWor1dBank)
    mynet.trainquery([wRivar,wBank],allurls,uRivar)
    mynet.trainquery([wWor1d],allurls,uEarth)

In [41]:
mynet.getresult([wWor1d,wBank],allurls)

[0.8705623942300412, 0.010855480106425808, 0.012084979924815572]

In [42]:
mynet.getresult([wRivar,wBank],allurls)

[-0.020212960144585894, 0.8852859851373398, 0.003354660406591399]

In [43]:
mynet.getresult([wBank],allurls)

[0.8749506589416379, 0.005900400330499424, -0.859269156107212]

In [44]:
mynet.getresult([wWor1d],allurls)

[-0.016145246792945437, 0.017987449223076896, 0.8508480187878236]